In [20]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import time
import pickle
from torch.utils.data import DataLoader, Dataset

In [21]:
import random
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True
# 设置随机数种子
setup_seed(7)

In [22]:
from file_utils.mydata import MyData

In [23]:
data_name = 'Cell_Phones_and_Accessories'

In [24]:
with open('input_data/dataset_time_'+data_name+'.bin', 'rb') as f:
    data_set = pickle.load(f)

In [25]:
'''
dataset statistic
'''
data_set.productNum, data_set.userNum, data_set.wordNum, len(data_set.train_data)

(10429, 27879, 82997, 172590)

In [26]:
# from models.DBML_time import PSM
from models.DBML_mut_time import PSM

## 定义参数 

In [27]:
'''
实验参数
'''
embedding_dim = 50
out_size = 10
batch_size = 512
neg_sample_num = data_set.neg_sample_num
dataLen = len(data_set.train_data)
batch_num = int(dataLen/batch_size)
full_len = batch_num*batch_size
time_bin_num = len(data_set.time_data)
total_epoch = 2
device  = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
# device  = torch.device("cpu")

## 定义模型

In [28]:
dbml = PSM(data_set.userNum,
           data_set.productNum,
           data_set.wordNum,
           embedding_dim,
           data_set.max_query_len,
           data_set.max_review_len,
           batch_size,
           data_set.time_num + 1,
           neg_num=5,
           sample_num=1,
           transfer_hidden_dim=100,
           sigma_parameter=1e-3,
           kl_parameter=1e-1,
           word_parameter=1e0,
           device=device)
dbml.to(device)

PSM(
  (time_embdding): Embedding(19, 50)
  (time2mean_u): Linear(in_features=100, out_features=50, bias=True)
  (time2mean_i): Linear(in_features=100, out_features=50, bias=True)
  (time2mean_w): Linear(in_features=100, out_features=50, bias=True)
  (time2std_i): Linear(in_features=100, out_features=50, bias=True)
  (time2std_u): Linear(in_features=100, out_features=50, bias=True)
  (time2std_w): Linear(in_features=100, out_features=50, bias=True)
  (user_mean): Embedding(27879, 50)
  (user_std): Embedding(27879, 50)
  (item_mean): Embedding(10429, 50)
  (item_std): Embedding(10429, 50)
  (wordEmbedding_mean): Embedding(82997, 50, padding_idx=0)
  (wordEmbedding_std): Embedding(82997, 50, padding_idx=0)
  (queryLinear): Linear(in_features=50, out_features=50, bias=True)
  (transfer_linear_u): Linear(in_features=50, out_features=100, bias=True)
  (transfer_linear_i): Linear(in_features=50, out_features=100, bias=True)
  (transfer_linear_ni): Linear(in_features=50, out_features=100, bia

In [29]:
# dbml = torch.nn.DataParallel(dbml)

### load model

In [30]:
# dbml.load_state_dict(torch.load('out/{}_{}_{}.pkl'.format( data_name, time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime()),'init')))

In [31]:
### 加载数据

In [32]:
# data_gen = DataLoader(data_set, batch_size=batch_size, shuffle=True, drop_last=True)

### 定义优化器

In [33]:
device  = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
dbml.to(device)
dbml.train()
optimizer = torch.optim.Adam(dbml.parameters(), lr=0.00005)

In [34]:
dbml.kl_parameter

0.1

In [35]:
# (uid, pid_pos, qids_pos, len_pos, text_ids, len_r, time_bin_pos)
# neg_item,neg_word
def get_time_batch(s, t):
    
    if (s+batch_size > len(data_set.time_data[t])):
        return None
    
    batch_data = [[torch.LongTensor(batch_size), torch.LongTensor(batch_size), torch.LongTensor(batch_size, data_set.max_query_len), torch.LongTensor(batch_size),
                torch.LongTensor(batch_size, data_set.max_review_len), torch.LongTensor(batch_size), torch.LongTensor(batch_size)],
                  [torch.LongTensor(batch_size, neg_sample_num), torch.LongTensor(batch_size, neg_sample_num)]]
    for i in range(batch_size):
        pos, neg = data_set.get_time_data(t, i+s)
#         print(pos)
#         print(neg)
        batch_data[0][0][i] = pos[0]
        batch_data[0][1][i] = pos[1]
        batch_data[0][2][i] = torch.tensor(pos[2])
        batch_data[0][3][i] = pos[3]
        batch_data[0][4][i] = torch.tensor(pos[4])
        batch_data[0][5][i] = pos[5]
        batch_data[0][6][i] = pos[6]
        batch_data[1][0][i] = torch.tensor(neg[0])
        batch_data[1][1][i] = torch.tensor(neg[1])
        
#         break
    return batch_data
# bt = get_time_batch(99,10)
# bt

In [17]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(comment='_cell_tr_0.1')

In [18]:
gs = 0

In [ ]:
full_time = data_set.time_num
total_epoch = 600
nann = 1
for t in range(full_time):
    total_batch = int(len(data_set.time_data[t])/batch_size)
    for e in range(total_epoch):
        for i in range(total_batch):
            data = get_time_batch(i*batch_size, t)
            if data is None:
                break
            user_mean, user_std, query, item_mean_pos, item_std_pos, items_mean_neg, items_std_neg,\
            user_sample, product_sample, product_sample_neg, loss, dis_pos, dis_neg, word_mean_pos,\
            word_std_pos, word_mean_neg, word_std_neg \
            = dbml(
            data[0][0].to(device), data[0][1].to(device),
            data[0][2].to(device), data[0][3].to(device),
            data[0][4].to(device), data[0][5].to(device), data[0][6].to(device),
            data[1][0].to(device),
            data[1][1].to(device))
        
            optimizer.zero_grad()
            loss[0].backward()
            optimizer.step()
            writer.add_scalar('Loss', loss[0].item(), global_step=gs)
#             writer.add_text('Process', 'T : {} | E: {}/{} | B: {}/{} | Loss: {} | POS: {} | NEG: {}'.format( t, e, total_epoch, i, total_batch, loss[0].item(), dis_pos.item(), dis_neg.item()), gs)
#             writer.add_text('Loss fac', 'Loss:{} | Main:{} | Word:{} | KL:{}'.format(loss[0].item(), loss[1].item(), loss[2].item(), loss[3].item()), gs)    
            gs+=1
            if (torch.isnan(loss[0])).item() == 1 and nann == 1:
                nann = 0
                writer.add_text('Process', 'T : {} | E: {}/{} | B: {}/{} | Loss: {} | POS: {} | NEG: {}'.format( t, e, total_epoch, i, total_batch, loss[0].item(), dis_pos.item(), dis_neg.item()), gs)
                writer.add_text('Nan Loss fac', 'Loss:{} | Main:{} | Word:{} | KL:{}'.format(loss[0].item(), loss[1].item(), loss[2].item(), loss[3].item()), gs)        
            if (i % 50 == 0):
                print('T : {} | E: {}/{} | B: {}/{} | Loss: {} | POS: {} | NEG: {}'.format( t, e, total_epoch, i, total_batch, loss[0].item(), dis_pos.item(), dis_neg.item()))
                print('Loss:{} | Main:{} | Word:{} | KL:{}'.format(loss[0].item(), loss[1].item(), loss[2].item(), loss[3].item()))


T : 11 | E: 0/600 | B: 0/1 | Loss: 238.40658569335938 | POS: 42.64954376220703 | NEG: 8.529852867126465
Loss:238.40658569335938 | Main:170.59844970703125 | Word:0.6477603912353516 | KL:671.6036376953125
T : 11 | E: 1/600 | B: 0/1 | Loss: 235.67852783203125 | POS: 42.30819320678711 | NEG: 8.461771011352539
Loss:235.67852783203125 | Main:169.23211669921875 | Word:0.6511571407318115 | KL:657.9524536132812
T : 11 | E: 2/600 | B: 0/1 | Loss: 232.97531127929688 | POS: 41.96697998046875 | NEG: 8.393803596496582
Loss:232.97531127929688 | Main:167.8658905029297 | Word:0.654581606388092 | KL:644.5482177734375
T : 11 | E: 3/600 | B: 0/1 | Loss: 230.3060302734375 | POS: 41.62779998779297 | NEG: 8.326130867004395
Loss:230.3060302734375 | Main:166.50836181640625 | Word:0.6580469608306885 | KL:631.396240234375
T : 11 | E: 4/600 | B: 0/1 | Loss: 227.6665802001953 | POS: 41.28966522216797 | NEG: 8.258552551269531
Loss:227.6665802001953 | Main:165.1555633544922 | Word:0.661536455154419 | KL:618.49487304

In [ ]:
writer.close()

### 模型保存

In [ ]:
import time as tt

In [ ]:
torch.save(dbml.state_dict(), 'out/{}_{}_{}.pkl'.format( data_name, tt.strftime("%Y-%m-%d-%H-%M-%S", tt.localtime()),'little_kl_lambda'))

In [26]:
e

399

### 测试模型

In [24]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.
def mean_reciprocal_rank(r):
    return np.sum(r / np.arange(1, r.size + 1))
def hit_rate(r):
    if (np.sum(r) >= 0.9):
        return 1
    else:
        return 0

In [25]:
def get_query_laten(q_linear, query, query_len, max_query_len):
    '''
    input size: (batch, maxQueryLen)
    对query处理使用函数
    tanh(W*(mean(Q))+b)
    '''
    query_len = torch.tensor(query_len).view(1,-1).float()
    # size: ((batch, maxQueryLen))) ---> (batch, len(query[i]), embedding)
    # query len mask 使得padding的向量为0
    len_mask = torch.tensor([ [1.]*int(i.item())+[0.]*(max_query_len-int(i.item())) for i in query_len]).unsqueeze(2)
    query = query.mul(len_mask)
    query = query.sum(dim=1).div(query_len)
    query = q_linear(query).tanh()

    return query

In [33]:
dbml.eval()
device=torch.device('cpu')
dbml.to(device)
all_p_m = torch.empty(data_set.time_num, data_set.productNum, embedding_dim)
for ii in range(data_set.time_num):
    for i in range(data_set.productNum):
        p_mean = dbml.item_mean(torch.tensor([i], device=device)).squeeze(1)
        time= dbml.time_embdding(torch.tensor([ii], device=device)).squeeze(1)
        p_mean = dbml.time2mean_i(torch.cat([p_mean, time], 1)).squeeze()
        all_p_m[ii][i] = p_mean

In [34]:
from tqdm import trange

In [36]:
eval_dataset = data_set.test_data
test_counter = 0
all_hr = 0
all_ndcg = 0
all_mrr = 0
for ii in trange(len(eval_dataset)):
    td = eval_dataset[ii]
    '''
    应该定义一个训练过的user， 这里简单的先取训练过的时间段的用户
    '''
    if (td[6] >= 0):
        user = dbml.user_mean(torch.tensor([td[0]], device=device)).squeeze(1)
        time= dbml.time_embdding(torch.tensor([td[6]], device=device)).squeeze(1)
        user = dbml.time2mean_u(torch.cat([user, time], 1)).squeeze()
        
        query_len = td[3]
        query = dbml.wordEmbedding_mean(torch.tensor([td[2]]))
        query = query.squeeze()
#         query = torch.cat(tuple([dbml.wordEmbedding_mean(torch.tensor([i], device=device).squeeze(0)) for i in td[2]])).view(1,-1,embedding_dim)
        query = get_query_laten(dbml.queryLinear, query, query_len, data_set.max_query_len)
        user_query = user+query
#         uq_i = torch.empty(datasets.productNum)
        user_query.squeeze_(0)
        uq_i = (user_query - all_p_m[td[6]]).norm(2, dim=1)*(-1.)
#         for i in range(datasets.productNum):
#             p_mean = product_time_latent[td[6]+1][i][0]
#             uq_i[i] = -1*(user_query-p_mean).norm(2).item()
        ranks_order = uq_i.topk(20)[1]
        r = torch.eq(ranks_order, td[1]).numpy()
        all_hr += hit_rate(r)
        all_mrr += mean_reciprocal_rank(r)
        all_ndcg += dcg_at_k(r, 20, 1)
        test_counter += 1
hr = all_hr / float(test_counter+1e-6)
mrr = all_mrr / float(test_counter+1e-6)
ndcg = all_ndcg / float(test_counter+1e-6)
print(hr, mrr, ndcg)


100%|██████████| 19403/19403 [01:07<00:00, 288.79it/s]

0.17466371178814288 0.05003045085297606 0.07721213340014016


In [ ]:
len(eval_dataset)

In [ ]:
data_set.eval_data

In [3]:
HasBN = 1

In [ ]:
dbml.eval()
device=torch.device('cpu')
dbml.device = device
dbml.to(device)
all_p_m = torch.empty(data_set.time_num, data_set.productNum, embedding_dim)
for ii in range(data_set.time_num):
    for i in range(data_set.productNum):
        p_mean = dbml.item_mean(torch.tensor([i], device=device)).squeeze(1)
        time= dbml.time_embdding(torch.tensor([ii], device=device)).squeeze(1)
        p_mean = dbml.time2mean(torch.cat([p_mean, time], 1))
        if (HasBN == 1):
            p_mean =dbml.itemTimeMeanBN(p_mean)
#         print(p_mean)
        
        p_std = dbml.item_std(torch.tensor([i], device=device)).squeeze(1).mul(0.5).exp()
        p_std = dbml.time2mean(torch.cat([p_std, time], 1))
#         print(p_std)
        if (HasBN == 1):
            p_std =dbml.itemTimeStdBN(p_std)
        pp = dbml.reparameter(p_mean, p_std)
#         print(pp)
#         print("##")
        all_p_m[ii][i] = pp
#         break

In [ ]:
eval_dataset = data_set.train_data
test_counter = 0
all_hr = 0
all_ndcg = 0
all_mrr = 0
for ii in trange(len(eval_dataset)):
    td = eval_dataset[ii]
    '''
    应该定义一个训练过的user， 这里简单的先取训练过的时间段的用户
    '''
    if (td[6] >= 13):
        user = dbml.user_mean(torch.tensor([td[0]], device=device)).squeeze(1)
        u_std = dbml.user_std(torch.tensor([td[0]], device=device)).squeeze(1)
        
        time= dbml.time_embdding(torch.tensor([td[6]], device=device)).squeeze(1)
        user = dbml.time2mean(torch.cat([user, time], 1))
        u_std = dbml.time2std(torch.cat([u_std, time], 1)).mul(0.5).exp()
        user = dbml.reparameter(user, u_std)
        if (HasBN == 1):
            user = dbml.userTimeMeanBN(user)
        query_len = td[3]
        query = dbml.wordEmbedding_mean(torch.tensor([data_set.test_data[1][2]]))
        if (HasBN == 1):
            query = dbml.queryMeanBN(query)
        query = query.squeeze()
        query = get_query_laten(dbml.queryLinear, query, query_len, data_set.max_query_len)
        user_query = user+query
#         uq_i = torch.empty(datasets.productNum)
        user_query.squeeze_(0)
        uq_i = (user_query - all_p_m[td[6]]).norm(2, dim=1)*(-1.)
        
        print(td[1])
        print(uq_i[td[1]])
        print(uq_i[3])
        print(uq_i[38])
        print(uq_i[378])
        break
#         for i in range(datasets.productNum):
#             p_mean = product_time_latent[td[6]+1][i][0]
#             uq_i[i] = -1*(user_query-p_mean).norm(2).item()
        ranks_order = uq_i.topk(20)[1]
        r = torch.eq(ranks_order, td[1]).numpy()
        all_hr += hit_rate(r)
        all_mrr += mean_reciprocal_rank(r)
        all_ndcg += dcg_at_k(r, 20, 1)
        test_counter += 1
hr = all_hr / float(test_counter+1e-6)
mrr = all_mrr / float(test_counter+1e-6)
ndcg = all_ndcg / float(test_counter+1e-6)
print(hr, mrr, ndcg)